In [19]:
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
import pickle


In [8]:
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')
stopwords = list(STOP_WORDS)

In [58]:
def text_summarizer(raw_docx, N):
    raw_text = raw_docx
    docx = nlp(raw_text)    
    
    # Build Word Frequency
    # word.text is tokenization in spacy
    word_frequencies = {}  
    for word in docx:  
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
    maximum_frequncy = max(word_frequencies.values())
    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    
    # Sentence Tokens
    sentence_list = [ sentence for sentence in docx.sents ]
    # Calculate Sentence Score and Ranking
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

    # Find N Largest
    summary_sentences = nlargest(N, sentence_scores, key=sentence_scores.get)
    ordered_summary_sentences = sorted(summary_sentences, key=lambda sentence: raw_docx.find(str(sentence)))
    final_sentences = [ w.text for w in ordered_summary_sentences ]
    summary = ' '.join(final_sentences)
    return summary
#     print("Original Document\n")
#     print(raw_docx)
#     print("Total Length:",len(raw_docx))
#     print('\n\nSummarized Document\n')
#     print(summary)
#     print("Total Length:",len(summary))

In [11]:
with open("../../data/disease_context_from_dev.pickle", "rb") as f:
    data = pickle.load(f)

In [101]:
N = 25
docx = data[567]
summary = text_summarizer(docx, N)

In [102]:
len(docx)

25518

In [103]:
print(docx)

Following is the contextual information about the Disease Smith-Magenis syndrome (Disease Ontology identifier of this Disease is DOID:0060768).
Smith-Magenis syndrome ISA Disease chromosomal deletion syndrome. Attributes of this relationship are:
(1) 'source' is Disease Ontology 
Smith-Magenis syndrome ASSOCIATES Gene RAI1. Attributes of this relationship are:
(1) 'sources' is ['DISEASES', 'OMIM'] (2) 'diseases_confidences' is [3.733, 5.0] (3) 'o_inheritance' is AD (4) 'diseases_scores' is ['7.465', 'CURATED'] 
Smith-Magenis syndrome PRESENTS Symptom Intellectual Disability. Attributes of this relationship are:
(1) 'enrichment' is 33.38706235253667 (2) 'odds' is 70.84369926234643 (3) 'source' is NCBI PubMed (4) 'cooccur' is 15 (5) 'fisher' is 3.668791085044043e-20 
Smith-Magenis syndrome PRESENTS Symptom Hoarseness. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-Magenis syndrome PRESENTS Symptom Seizures. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-

In [104]:
len(summary)

3282

In [105]:
print(summary)

Attributes of this relationship are:
(1) 'sources' is ['DISEASES', 'OMIM'] (2) ' diseases_confidences' is [3.733, 5.0] (3) 'o_inheritance' is AD (4) 'diseases_scores' is ['7.465', 'CURATED'] 
Smith-Magenis syndrome PRESENTS Symptom Intellectual Disability. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-Magenis syndrome PRESENTS Symptom Abnormality of the kidney. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-Magenis syndrome PRESENTS Symptom Abnormality of the thyroid gland. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-Magenis syndrome PRESENTS Symptom Abnormality of the cardiovascular system. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-Magenis syndrome PRESENTS Symptom Abnormality of the cardiovascular system. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-Magenis syndrome PRESENTS Symptom Self-mutilation. Attributes of this relationship are:
(1) 'source' is HPO 
Smith-Magenis syndrome PRESENTS S

In [42]:
import pyperclip

pyperclip.copy(docx)

In [43]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn", cache_dir=CACHE_DIR)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn", cache_dir=CACHE_DIR)

inputs = tokenizer(docx, max_length=1024, return_tensors="pt")
summary_ids = model.generate(inputs["input_ids"], do_sample=False, min_length=600, max_length=2500)
summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]




NameError: name 'TextStreamer' is not defined